In [ ]:
!pip list

In [2]:
from langchain_community.document_loaders import PyPDFLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os, pickle

PDF_PATH = "BOOK.pdf"
SAVE_DIR = "../faiss_layer1"

os.makedirs(SAVE_DIR, exist_ok=True)

# Load PDF
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()

# Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)
chunks = text_splitter.split_documents(documents)

# Embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Build FAISS
vectorstore = FAISS.from_documents(chunks, embedding_model)

# Save FAISS index
vectorstore.save_local(SAVE_DIR)

print(f"✅ Layer 1 built with {len(chunks)} chunks")

C:\Users\LS\AppData\Local\Temp\ipykernel_3616\1851037292.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 177.32it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Layer 1 built with 23 chunks


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
import os, pickle, faiss
import numpy as np

LAYER1_DIR = "../faiss_layer1"
LAYER2_DIR = "../faiss_layer2"
os.makedirs(LAYER2_DIR, exist_ok=True)

# Load Layer 1
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.load_local(
    LAYER1_DIR,
    embedding_model,
    allow_dangerous_deserialization=True
)

raw_docstore = vectorstore.docstore._dict

# LLM
llm = Ollama(model="gemma3:4b")

summary_texts = []

print(f"Total chunks: {len(raw_docstore)}\n")

for i, (doc_id, doc) in enumerate(raw_docstore.items(), start=1):
    try:
        summary = llm.invoke(
            f"Summarize this text factually without adding new information:\n{doc.page_content}"
        )
        summary_texts.append({
            "id": doc_id,
            "text": summary
        })
        print(f"✅ {i} summarized")
    except Exception as e:
        print(f"❌ {i} failed: {e}")

# Embed summaries
summary_vectors = embedding_model.embed_documents(
    [s["text"] for s in summary_texts]
)

dimension = len(summary_vectors[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(summary_vectors).astype("float32"))

# Save metadata
docstore = {s["id"]: s["text"] for s in summary_texts}
index_to_docstore_id = {i: s["id"] for i, s in enumerate(summary_texts)}

with open(f"{LAYER2_DIR}/index.pkl", "wb") as f:
    pickle.dump(
        {"docstore": docstore, "mapping": index_to_docstore_id},
        f
    )

faiss.write_index(index, f"{LAYER2_DIR}/index.faiss")

print("\n🎉 Layer 2 built successfully")


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 235.50it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
C:\Users\LS\AppData\Local\Temp\ipykernel_3616\2592347277.py:25: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:4b")


Total chunks: 23

✅ 1 summarized
✅ 2 summarized
✅ 3 summarized
✅ 4 summarized
✅ 5 summarized
✅ 6 summarized
✅ 7 summarized
✅ 8 summarized
✅ 9 summarized
✅ 10 summarized
✅ 11 summarized
✅ 12 summarized
✅ 13 summarized
✅ 14 summarized
✅ 15 summarized
✅ 16 summarized
✅ 17 summarized
✅ 18 summarized
✅ 19 summarized
✅ 20 summarized
✅ 21 summarized
✅ 22 summarized
✅ 23 summarized

🎉 Layer 2 built successfully


In [6]:
import os
import numpy as np
import pickle, faiss
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama

# Load embeddings + LLM
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
llm = Ollama(model="gemma3:4b")

BASE_DIR = os.path.dirname(os.path.abspath(r"C:/Users/LS/RAG-based-on-PDF/faiss_layer2"))

FAISS_INDEX_PATH = os.path.join(
    BASE_DIR, "faiss_layer2", "index.faiss"
)

FAISS_META_PATH = os.path.join(
    BASE_DIR, "faiss_layer2", "index.pkl"
)

index = faiss.read_index(FAISS_INDEX_PATH)

with open(FAISS_META_PATH, "rb") as f:
    data = pickle.load(f)


docstore = data["docstore"]
mapping = data["mapping"]

def ask(question, top_k=3):
    q_vec = embedding_model.embed_query(question)
    distances, indices = index.search(
        np.array([q_vec]).astype("float32"),
        top_k
    )

    context = "\n\n".join(
        docstore[mapping[i]] for i in indices[0]
    )

    prompt = f"""
Answer the question ONLY using the context below.
If the answer is not present, say "Not found in the document".

Context:
{context}

Question:
{question}
"""

    return llm.invoke(prompt)

# Example
print(ask("What is this documentation is all about?"))

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 190.60it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


VS Code features, including configuring settings for specific folders, defining debug configurations, and exploring workspaces.
